<a href="https://colab.research.google.com/github/valefe/playground_transformers/blob/main/Byte_Pair_Encoding_Before_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
!pip install bpe
!pip install transformers

In [67]:
from torch import nn, tensor
import pandas as pd
import numpy as np

In [68]:
df = pd.read_csv("description_bisac.csv", nrows=5000, names=["id", "mainDescription", "bisacCode"])
# TO-DO: Mask some words

# Tokenizing the data

In [69]:
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = Whitespace()

In [70]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.train(["./description_bisac.csv"], trainer=trainer)

*"Quando você quer alguma coisa, todo o Universo conspira para que você realize seu desejo. De tempos em tempos, surge um livro que muda para sempre a vida de seus leitores. O Alquimista é um deles. Com mais de 65 milhões de exemplares vendidos no mundo inteiro, o mais famoso título de Paulo Coelho já se estabeleceu como um clássico moderno, atemporal e universal. Quase 25 anos após seu lançamento, segue fascinando públicos cada vez maiores, de diferentes gerações. Simples, sábia e inspiradora, esta história refaz os passos de um pastor da Andaluzia que viaja para o deserto egípcio em busca de um tesouro enterrado nas Pirâmides. O que começa como uma jornada para encontrar bens mate¬riais torna-se uma descoberta das riquezas que escondemos dentro de nós mesmos. As belas lições que Santiago aprende pelo caminho nos falam da sabedoria de ouvir o que diz nosso coração, ler os sinais com que deparamos ao longo da vida e, acima de tudo, seguir os nossos sonhos."*

In [80]:
o_alquimista = "Quando você quer alguma coisa, todo o Universo conspira para que você realize seu desejo. De tempos em tempos, surge um livro que muda para sempre a vida de seus leitores. O Alquimista é um deles. Com mais de 65 milhões de exemplares vendidos no mundo inteiro, o mais famoso título de Paulo Coelho já se estabeleceu como um clássico moderno, atemporal e universal. Quase 25 anos após seu lançamento, segue fascinando públicos cada vez maiores, de diferentes gerações. Simples, sábia e inspiradora, esta história refaz os passos de um pastor da Andaluzia que viaja para o deserto egípcio em busca de um tesouro enterrado nas Pirâmides. O que começa como uma jornada para encontrar bens mate¬riais torna-se uma descoberta das riquezas que escondemos dentro de nós mesmos. As belas lições que Santiago aprende pelo caminho nos falam da sabedoria de ouvir o que diz nosso coração, ler os sinais com que deparamos ao longo da vida e, acima de tudo, seguir os nossos sonhos."
encoded = tokenizer.encode(o_alquimista)
encoded.ids[:10]


[2800, 443, 1336, 122, 62, 140, 105, 1993, 11, 860]

# Creating a Dataset 

In [72]:
from torch.utils.data import Dataset, DataLoader

In [97]:
class MetabooksDataset(Dataset):
    """Metabooks dataset."""

    def __init__(self, csv_file, tokenizer):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            tokenizer (huggingface Tokenizer): Will encode str sentences
        """
        self.genres_df = pd.read_csv("description_bisac.csv", names=["id", "mainDescription", "bisacCode"])
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        encoded = self.tokenizer.encode(self.genres_df["mainDescription"][idx])
        label = self.genres_df["bisacCode"].astype("category").cat.codes[idx]
        print(dir(encoded))
        print(tensor(encoded.ids))
        return {"text": tensor(encoded.ids), "label": tensor(label)}

In [98]:
mb_dataset = MetabooksDataset('description_bisac.csv', tokenizer)

In [99]:
mb_dataset[0]

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', 'attention_mask', 'char_to_token', 'char_to_word', 'ids', 'merge', 'n_sequences', 'offsets', 'overflowing', 'pad', 'sequence_ids', 'set_sequence_id', 'special_tokens_mask', 'token_to_chars', 'token_to_sequence', 'token_to_word', 'tokens', 'truncate', 'type_ids', 'word_ids', 'word_to_chars', 'word_to_tokens', 'words']
tensor([ 607,  212,  103, 3895, 3954, 3983, 3993,  387,  140, 3967, 4008,   83,
         371,   13, 2015, 2014,   54, 4015,    9, 2596, 1321, 3982,   95, 2540,
        2547,    9, 2545, 1321, 2542,   55,  234, 2558,  141,   70,  745,  113,
        1155, 1690, 3986,  103,  231, 4006, 3975,   13, 3965, 4005,  123,   97,
  

{'label': tensor(0, dtype=torch.int8),
 'text': tensor([ 607,  212,  103, 3895, 3954, 3983, 3993,  387,  140, 3967, 4008,   83,
          371,   13, 2015, 2014,   54, 4015,    9, 2596, 1321, 3982,   95, 2540,
         2547,    9, 2545, 1321, 2542,   55,  234, 2558,  141,   70,  745,  113,
         1155, 1690, 3986,  103,  231, 4006, 3975,   13, 3965, 4005,  123,   97,
          130, 2605,   60,  169, 3973,  134, 2630,   83, 3933, 1701,  103,  700,
           70,  392,   56,  297,   11, 2389,  502,  567,   11,  274,  705,  117,
         1341,  831,   11,  180, 2541,  106, 2546,  106, 3990,   11,  239,  552,
          117,  433, 3935, 2656, 2015, 2014, 3979, 2019,   12,  134,  117,  633,
          584, 2465, 3991,  113, 3949,   11, 3969,   12,  134,  117, 2304,   60,
         3998,   13, 3994,  113, 3957,   31,   13, 3974, 3992, 2014,   11,  371,
           13, 2015, 1722,  552, 2541,  106, 2546,  106, 2619, 3995,   13, 3899,
         4013,  117, 4002, 1645,   11,  117, 2542,   11,  117,

# Fine Tuning

In [86]:
for i in range(200):
  data = mb_dataset[0]
  outputs = model(data["text"], labels=data["label"])
  loss = outputs.loss
  loss.backward()
  optimizer.step()

ValueError: ignored

In [76]:
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import BertForSequenceClassification  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads

model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased')

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

In [77]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, framework="pt")